In [17]:
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, save_img
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import numpy as np

In [18]:
# Путь до основного каталога
base_dir = './garbage_classification'

# Составление массива классов из папок
garbage_types = ['battery', 'biological', 'cardboard', 'metal', 'paper', 'plastic']

# сбор путей к изображениям и меткам
image_paths = []
image_labels =[]

for garbageType in garbage_types:
    garbage_dir = os.path.join(base_dir, garbageType) # ./garbage_classification/battery
    for namePhoto in os.listdir(garbage_dir):
        image_paths.append(os.path.join(garbage_dir, namePhoto))
        image_labels.append(garbageType)

# Создаем DataFrame: image_paths, image_labels
df = pd.DataFrame({
    'image_path': image_paths,
    'label': image_labels
})
df

,image_path,label
0,./garbage_classification\battery\battery1.jpg,battery
1,./garbage_classification\battery\battery10.jpg,battery
2,./garbage_classification\battery\battery100.jpg,battery
3,./garbage_classification\battery\battery101.jpg,battery
4,./garbage_classification\battery\battery102.jpg,battery
...,...,...
5500,./garbage_classification\plastic\plastic95.jpg,plastic
5501,./garbage_classification\plastic\plastic96.jpg,plastic
5502,./garbage_classification\plastic\plastic97.jpg,plastic
5503,./garbage_classification\plastic\plastic98.jpg,plastic


In [19]:
# Разделяем данные на тренировочную и тестовую выборки в соотношении 70 на 30 процентов для тренировочной и  тестовой соответственно
train_df, test_df = train_test_split(df, test_size=0.3, random_state=23)

In [6]:
# Аугментация данных
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=45,
    vertical_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
# Создаем генераторы
train_generator = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'image_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe = test_df,
    x_col = 'image_path',
    y_col = 'label',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 3853 validated image filenames belonging to 6 classes.
Found 1652 validated image filenames belonging to 6 classes.


In [8]:
# Содание модели
def create_model():
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))
    return model

In [9]:
optimizers = {
    'adam': 'adam',
    'adamw': 'adamw',
    'sgd': 'sgd'
}

final_accuracies = {}

for opt_name, opt in optimizers.items():
    print(f"Обучение с оптимизатором {opt_name}.")
    model = create_model()
    model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(
        train_generator,
        epochs = 10,
        validation_data=test_generator,
        verbose=1
    )

    final_accuracy = history.history['val_accuracy'][-1]
    final_accuracies[opt_name] = final_accuracy
    print(f"{opt_name}: Final accuracy = {final_accuracy:.4f}")

print("\nResults")
for opt_name, accuracy in final_accuracies.items():
    print(f"{opt_name}: {accuracy:.4f}")

Обучение с оптимизатором adam.


C:\Users\kseny\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\kseny\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 131s 1s/step - accuracy: 0.2988 - loss: 2.0380 - val_accuracy: 0.5339 - val_loss: 1.1975
Epoch 2/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 121s 995ms/step - accuracy: 0.5381 - loss: 1.2587 - val_accuracy: 0.5823 - val_loss: 1.1257
Epoch 3/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 124s 1s/step - accuracy: 0.5857 - loss: 1.1419 - val_accuracy: 0.6090 - val_loss: 1.0363
Epoch 4/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - accuracy: 0.6004 - loss: 1.0890 - val_accuracy: 0.6410 - val_loss: 0.9776
Epoch 5/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 128s 1s/step - accuracy: 0.6238 - loss: 1.0243 - val_accuracy: 0.6507 - val_loss: 0.9890
Epoch 6/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 127s 1s/step - accuracy: 0.6600 - loss: 0.9548 - val_accuracy: 0.6719 - val_loss: 0.9139
Epoch 7/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 125s 1s/step - accuracy: 0.6325 - loss: 1.0081 - val_accuracy: 0.6937 - val_loss: 0.8663
Epoch 8/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 128s 1s/step - accuracy: 0.6556 - loss: 0.9423 - val_a

In [12]:
new_train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

new_test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

Found 3853 validated image filenames belonging to 6 classes.
Found 1652 validated image filenames belonging to 6 classes.


In [13]:
optimizers = {
    'adam': 'adam',
    'adamw': 'adamw',
}

final_accuracies = {}

for opt_name, opt in optimizers.items():
    print(f"Обучение с оптимизатором {opt_name}.")
    model = create_model()
    model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(
        new_train_generator,
        epochs = 15,
        validation_data=new_test_generator,
        verbose=1
    )

    final_accuracy = history.history['val_accuracy'][-1]
    final_accuracies[opt_name] = final_accuracy
    print(f"{opt_name}: Final accuracy = {final_accuracy:.4f}")

print("\nResults")
for opt_name, accuracy in final_accuracies.items():
    print(f"{opt_name}: {accuracy:.4f}")

Обучение с оптимизатором adam.


C:\Users\kseny\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\kseny\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.3262 - loss: 2.0039 - val_accuracy: 0.5860 - val_loss: 1.1374
Epoch 2/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - accuracy: 0.5641 - loss: 1.1894 - val_accuracy: 0.6162 - val_loss: 1.0875
Epoch 3/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 112s 2s/step - accuracy: 0.5950 - loss: 1.1448 - val_accuracy: 0.6192 - val_loss: 1.0431
Epoch 4/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 112s 2s/step - accuracy: 0.6453 - loss: 1.0197 - val_accuracy: 0.6755 - val_loss: 0.9066
Epoch 5/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 112s 2s/step - accuracy: 0.6621 - loss: 0.9513 - val_accuracy: 0.6961 - val_loss: 0.8436
Epoch 6/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - accuracy: 0.6854 - loss: 0.9068 - val_accuracy: 0.6574 - val_loss: 0.9515
Epoch 7/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - accuracy: 0.6747 - loss: 0.8918 - val_accuracy: 0.7185 - val_loss: 0.8106
Epoch 8/15
61/61 ━━━━━━━━━━━━━━━━━━━━ 112s 2s/step - accuracy: 0.6883 - loss: 0.8655 - val_accuracy: 0.6574 - v